#1. Funciones auxiliares y lectura de datos


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

def leer_rango_excel_a_array(ruta_archivo, hoja_nombre=0, columnas_a_leer="", filas_a_leer=1):
    """
    Abre un archivo Excel y extrae un rango de celdas específico
    (D3 a BO42) y lo guarda en un array (lista de listas).
"""
    print(f"Intentando leer el archivo: {ruta_archivo}...")

    try:
        # 1. Filas:
        # Queremos empezar en la Fila 3.
        # Las filas en pandas se indexan desde 0.
        # Fila 1 = índice 0, Fila 2 = índice 1, Fila 3 = índice 2.
        # Así que saltamos las 2 primeras filas.
        filas_a_saltar = 3  # (Saltamos Fila 1 :texto Fila 2 : calibracion y Fila 3: checkeo salon vs dormitorio)


        #filas_a_leer = 40
        # Queremos leer desde la fila 3 hasta la 42 (inclusive).
        # Número de filas = 42 - 3 + 1 = 40 filas.

        # --- Lectura del archivo ---

        # Usamos 'header=None' porque el rango (D3) no es un encabezado,
        # es el inicio de los datos.
        df_rango = pd.read_excel(
            ruta_archivo,
            sheet_name= hoja_nombre, #Segunda hoja, Espectro Total
            header=None,             # Trata la primera fila leída (Fila 3) como datos
            skiprows=filas_a_saltar, # Salta las 2 primeras filas
            usecols=columnas_a_leer, # Lee solo el rango de columnas D-BO
            nrows=filas_a_leer       # Lee exactamente 40 filas
        )

        # Convertir el DataFrame de pandas a un array (lista de listas)
        # .values obtiene los datos como un array de NumPy
        # .tolist() lo convierte a una lista estándar de Python
        array_valores = df_rango.values.tolist()

        # Verificación
        if array_valores:
            num_filas = len(array_valores)
            num_cols = len(array_valores[0]) # Longitud de la primera fila
            print(f"Éxito: Se leyó un array de {num_filas} filas x {num_cols} columnas.")
        else:
            print("Advertencia: El rango D3:BO42 parece estar vacío.")

        return array_valores

    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo en la ruta: '{ruta_archivo}'.")
        return None
    except Exception as e:
        print(f"Ocurrió un error inesperado al leer el rango: {e}")
        return None

def separar_filas_en_matrices(datos_array):
    """
    Toma una lista de listas (matriz) y la separa en dos matrices:
    - Una con las filas de índice par (0, 2, 4...)
    - Otra con las filas de índice impar (1, 3, 5...)
    """

    # Listas para guardar las *filas* (que son listas)
    matriz_filas_impares_excel = []
    matriz_filas_pares_excel = []

    for i, fila in enumerate(datos_array):

        # i % 2 == 0 significa índice 0, 2, 4... (Filas Excel 3, 5, 7...)
        if i % 2 == 0:
            # --- CAMBIO CLAVE ---
            # Añade la 'fila' (que es una lista) como un nuevo elemento
            matriz_filas_impares_excel.append(fila)

        # else significa índice 1, 3, 5... (Filas Excel 4, 6, 8...)
        else:
            # --- CAMBIO CLAVE ---
            matriz_filas_pares_excel.append(fila)

    return matriz_filas_impares_excel, matriz_filas_pares_excel


In [3]:
#LO PRIMERO: ABRIR LOS ARCHIVOS

archivo_S1= "P3_G7_S1_Datos.xlsx"
archivo_S2= "P3_G7_S2_Datos.xlsx"

datos_sonometro_1 = leer_rango_excel_a_array(archivo_S1, "Banda Ancha Total", "U" , 24)
datos_sonometro_2 = leer_rango_excel_a_array(archivo_S2, "Banda Ancha Total", "U" , 48)

#SEPARAR POR PUNTOS A B Y C Y LUEGO RUIDOS DE FONDO

# Filtra los índices 3, 4, 5 de cada bloque de 6
datos_A = [x for i, x in enumerate(datos_sonometro_2) if i % 6 >= 3]

# Filtra los índices 0, 1, 2 de cada bloque de 6
datos_C = [x for i, x in enumerate(datos_sonometro_2) if i % 6 < 3]

ruido_A = np.array(datos_A[:3]) #QUITAR LAS TRES PRIMERAS POSICIONES QUE SON DE RUIDO DE FONDO
medidas_A = np.array(datos_A[3:])

ruido_B = np.array(datos_sonometro_1[:3]) #RUIDO EN B
medidas_B = np.array(datos_sonometro_1[3:]) #MEDIDAS EN B

ruido_C = np.array(datos_C[:3])
medidas_C = np.array(datos_C[3:])

#print(ruido_A, ruido_B, ruido_C) PARA COMPROBAR QUE FUNCIONE

#CORRECION RUIDO DE FONDO: PROMEDIADO DEL RUIDO DE FONDO ENERGETICO

ruido_A_promed = np.round(10 * np.log10((1/len(ruido_A)) * np.sum(10**(0.1*ruido_A))), 1)
ruido_B_promed = np.round(10 * np.log10((1/len(ruido_B)) * np.sum(10**(0.1*ruido_B))), 1)
ruido_C_promed = np.round(10 * np.log10((1/len(ruido_C)) * np.sum(10**(0.1*ruido_C))), 1)

#print(ruido_A_promed,ruido_B_promed, ruido_C_promed) #TUDO VA BEM

#print(len(medidas_A))
#AHORA VOY A SEPARAR LAS FASES

fases_A = np.split(medidas_A,7) #En este array están las medidas de cada fase, SIN PROMEDIAR
fases_B = np.split(medidas_B,7) #En este array están las medidas de cada fase, SIN PROMEDIAR
fases_C = np.split(medidas_C,7) #En este array están las medidas de cada fase, SIN PROMEDIAR


#print(fases_A[0]) Te devuelve los tres valores de la FASE 1 en la posición A

fases_A_promed = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_A
])

fases_B_promed = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_B
])

fases_C_promed = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_C
])

#TODO: COMPROBAR EL CRITERIO EN CADA FASE DE QUE EL RUIDO DE FONDO SEA 3dB INFERIOR A LA MEDIDA DE LA FASE. SE REALIZA SOBRE EL PROMEDIADO DE LAS TRES MEDIDAS EN CADA PUNTO
#EJEMPLO : FASES_A_PROMED[0]-RUIDO_A_PROMED PARA COMPROBAR EN LA FASE 1
#A INVESTIGAR ANTES: ES POSIBLE QUE HAYA QUE HACER ANTES EL PROMEDIADO DE LOS PUNTOS ABC PARA COMPROBAR ESTO CONTRA EL RUIDO ABC? LA INTUICION ME DICE QUE NO, QUE ES PUNTO A PUNTO

#Comprobación de cada punto, cada fase si cumple el criterio de los 3dB

ruido_promed = np.round(10 * np.log10((1/3) * ((10**(0.1*ruido_A_promed)) + (10**(0.1*ruido_B_promed)) + (10**(0.1*ruido_C_promed)))), 1)
fases_promed = np.round(10 * np.log10((1/3) * ((10**(0.1*fases_A_promed)) + (10**(0.1*fases_B_promed)) + (10**(0.1*fases_C_promed)))), 1)

diferencias = fases_promed - ruido_promed
print(diferencias)

#LAS FASES QUE NO CUMPLEN SON LAS 5, 6 Y 7. EN ELLAS NO SE PUEDE CORREGIR POR RUIDO DE FONDO Y LOS RESULTADOS SE DARÁN COMO COTAS

fases_1234 = fases_promed[:4]
fases_567 = fases_promed[4:7]

fases_1234_corregidas = np.round(10 * np.log10(((10**(0.1*fases_1234)) - (10**(0.1*ruido_promed)))), 1)

La_fases =  np.concatenate((fases_1234_corregidas , fases_567) , axis = 0)

print(fases_promed, La_fases)

Intentando leer el archivo: P3_G7_S1_Datos.xlsx...
Éxito: Se leyó un array de 24 filas x 1 columnas.
Intentando leer el archivo: P3_G7_S2_Datos.xlsx...


/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Éxito: Se leyó un array de 48 filas x 1 columnas.
[3.9 6.3 6.6 3.7 2.  0.5 0.9]
[43.3 45.7 46.  43.1 41.4 39.9 40.3] [41.  44.5 44.9 40.7 41.4 39.9 40.3]


In [4]:
#Importamos la columna correspondiente a LCeq,T

datos_sonometro_1_LCeq = leer_rango_excel_a_array(archivo_S1, "Banda Ancha Total", "X" , 24)
datos_sonometro_2_LCeq = leer_rango_excel_a_array(archivo_S2, "Banda Ancha Total", "X" , 48)

datos_A_LCeq = [x for i, x in enumerate(datos_sonometro_2_LCeq) if i % 6 >= 3]


datos_C_LCeq = [x for i, x in enumerate(datos_sonometro_2_LCeq) if i % 6 < 3]

ruido_A_LCeq = np.array(datos_A_LCeq[:3])
medidas_A_LCeq = np.array(datos_A_LCeq[3:])

ruido_B_LCeq = np.array(datos_sonometro_1_LCeq[:3])
medidas_B_LCeq = np.array(datos_sonometro_1_LCeq[3:])

ruido_C_LCeq = np.array(datos_C_LCeq[:3])
medidas_C_LCeq = np.array(datos_C_LCeq[3:])


ruido_A_LCeq_promed = np.round(10 * np.log10((1/len(ruido_A_LCeq)) * np.sum(10**(0.1*ruido_A_LCeq))), 1)
ruido_B_LCeq_promed = np.round(10 * np.log10((1/len(ruido_B_LCeq)) * np.sum(10**(0.1*ruido_B_LCeq))), 1)
ruido_C_LCeq_promed = np.round(10 * np.log10((1/len(ruido_C_LCeq)) * np.sum(10**(0.1*ruido_C_LCeq))), 1)


fases_A_LCeq = np.split(medidas_A_LCeq,7)
fases_B_LCeq = np.split(medidas_B_LCeq,7)
fases_C_LCeq = np.split(medidas_C_LCeq,7)


fases_A_promed_LCeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_A_LCeq
])

fases_B_promed_LCeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_B_LCeq
])

fases_C_promed_LCeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_C_LCeq
])



ruido_promed_LCeq = np.round(10 * np.log10((1/3) * ((10**(0.1*ruido_A_LCeq_promed)) + (10**(0.1*ruido_B_LCeq_promed)) + (10**(0.1*ruido_C_LCeq_promed)))), 1)
fases_promed_LCeq = np.round(10 * np.log10((1/3) * ((10**(0.1*fases_A_promed_LCeq)) + (10**(0.1*fases_B_promed_LCeq)) + (10**(0.1*fases_C_promed_LCeq)))), 1)

print(ruido_promed_LCeq)
print(fases_promed_LCeq)

diferencias_LCeq = fases_promed_LCeq - ruido_promed_LCeq
print("Valores de LCeq - LAeq para cada fase de ruido: ",diferencias_LCeq) #NO CUMPLE NINGUNA, FULL COTAS

LCeq_fases = fases_promed_LCeq
print("Valores de LCeq con correccion de RF para cada fase de ruido: ",LCeq_fases)

Lf =  LCeq_fases - La_fases
print("Valores de Lf para cada fase de ruido: ",Lf)

Kf = [0,0,0,0,0,0,0] #TODOS LOS VALORES SUPERAN 15, POR TANTO TODAS LAS Kf SON 6 dB
print("Valores de Kf para cada fase de ruido: ",Kf)



Intentando leer el archivo: P3_G7_S1_Datos.xlsx...
Éxito: Se leyó un array de 24 filas x 1 columnas.
Intentando leer el archivo: P3_G7_S2_Datos.xlsx...


/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Éxito: Se leyó un array de 48 filas x 1 columnas.
60.1
[60.2 60.8 60.5 59.9 60.2 60.  59.8]
Valores de LCeq - LAeq para cada fase de ruido:  [ 0.1  0.7  0.4 -0.2  0.1 -0.1 -0.3]
Valores de LCeq con correccion de RF para cada fase de ruido:  [60.2 60.8 60.5 59.9 60.2 60.  59.8]
Valores de Lf para cada fase de ruido:  [19.2 16.3 15.6 19.2 18.8 20.1 19.5]
Valores de Kf para cada fase de ruido:  [0, 0, 0, 0, 0, 0, 0]


In [5]:
#Importamos la columna correspondiente a LAIeq,T

datos_sonometro_1_LAIeq = leer_rango_excel_a_array(archivo_S1, "Banda Ancha Total", "S" , 24)
datos_sonometro_2_LAIeq = leer_rango_excel_a_array(archivo_S2, "Banda Ancha Total", "S" , 48)



datos_A_LAIeq = [x for i, x in enumerate(datos_sonometro_2_LAIeq) if i % 6 >= 3]


datos_C_LAIeq = [x for i, x in enumerate(datos_sonometro_2_LAIeq) if i % 6 < 3]

ruido_A_LAIeq = np.array(datos_A_LAIeq[:3])
medidas_A_LAIeq = np.array(datos_A_LAIeq[3:])

ruido_B_LAIeq = np.array(datos_sonometro_1_LAIeq[:3])
medidas_B_LAIeq = np.array(datos_sonometro_1_LAIeq[3:])

ruido_C_LAIeq = np.array(datos_C_LAIeq[:3])
medidas_C_LAIeq = np.array(datos_C_LAIeq[3:])


ruido_A_LAIeq_promed = np.round(10 * np.log10((1/len(ruido_A_LAIeq)) * np.sum(10**(0.1*ruido_A_LAIeq))), 1)
ruido_B_LAIeq_promed = np.round(10 * np.log10((1/len(ruido_B_LAIeq)) * np.sum(10**(0.1*ruido_B_LAIeq))), 1)
ruido_C_LAIeq_promed = np.round(10 * np.log10((1/len(ruido_C_LAIeq)) * np.sum(10**(0.1*ruido_C_LAIeq))), 1)


fases_A_LAIeq = np.split(medidas_A_LAIeq,7)
fases_B_LAIeq = np.split(medidas_B_LAIeq,7)
fases_C_LAIeq = np.split(medidas_C_LAIeq,7)


fases_A_promed_LAIeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_A_LAIeq
])

fases_B_promed_LAIeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_B_LAIeq
])

fases_C_promed_LAIeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_C_LAIeq
])


ruido_promed_LAIeq = np.round(10 * np.log10((1/3) * ((10**(0.1*ruido_A_LAIeq_promed)) + (10**(0.1*ruido_B_LAIeq_promed)) + (10**(0.1*ruido_C_LAIeq_promed)))), 1)
fases_promed_LAIeq = np.round(10 * np.log10((1/3) * ((10**(0.1*fases_A_promed_LAIeq)) + (10**(0.1*fases_B_promed_LAIeq)) + (10**(0.1*fases_C_promed_LAIeq)))), 1)

diferencias_LAIeq = fases_promed_LAIeq - ruido_promed_LAIeq

print("Valores de LAIeq - LAIeqRF para cada fase de ruido: ",diferencias_LAIeq) #CUMPLEN 1,2,3; NO CUMPLEN 4,5,6,7

fases_123 = fases_promed_LAIeq[:3]
fases_4567 = fases_promed_LAIeq[3:7]

fases_123_corregidas = np.round(10 * np.log10(((10**(0.1*fases_123)) - (10**(0.1*ruido_promed_LAIeq)))), 1)

LAIeq_fases =  np.concatenate((fases_123_corregidas , fases_4567) , axis = 0)
print("Valores de LAIeq con correccion de RF para cada fase de ruido: ",LAIeq_fases)

Li =  LAIeq_fases - La_fases
print("Valores de Li para cada fase de ruido: ",Li)

Ki = [0,3,3,0,0,0,0] #LOS QUE VALEN 6 ES POR LAS COTAS, EL RESTO ESTÁN TODOS POR DEBAJO DE 10, LUEGO SU Ki VALE 0
print("Valores de Ki para cada fase de ruido: ",Ki)

#TODO QUE ALGUIEN REVISE LO DE LAS COTAS, YO LE VEO SENTIDO PERO ECHADLE UN OJO POR CONTRASTAR

Intentando leer el archivo: P3_G7_S1_Datos.xlsx...
Éxito: Se leyó un array de 24 filas x 1 columnas.
Intentando leer el archivo: P3_G7_S2_Datos.xlsx...


/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Éxito: Se leyó un array de 48 filas x 1 columnas.
Valores de LAIeq - LAIeqRF para cada fase de ruido:  [ 5.2 13.3 13.4  2.9  0.9 -0.8 -0.2]
Valores de LAIeq con correccion de RF para cada fase de ruido:  [45.7 55.2 55.3 45.  43.  41.3 41.9]
Valores de Li para cada fase de ruido:  [ 4.7 10.7 10.4  4.3  1.6  1.4  1.6]
Valores de Ki para cada fase de ruido:  [0, 3, 3, 0, 0, 0, 0]


CORRECCION TONAL

In [32]:
Espectro_total_s1 = leer_rango_excel_a_array(archivo_S1, "Espectro Total", "EC:FF" , 24)
Espectro_total_s2 = leer_rango_excel_a_array(archivo_S2, "Espectro Total", "EC:FF" , 48)

datos_A_espectro = [x for i, x in enumerate(Espectro_total_s2) if i % 6 >= 3]
datos_C_espectro = [x for i, x in enumerate(Espectro_total_s2) if i % 6 < 3]

ruido_A_espectro = np.array(datos_A_espectro[:3])
medidas_A_espectro = np.array(datos_A_espectro[3:])

ruido_B_espectro = np.array(Espectro_total_s1[:3])
medidas_B_espectro = np.array(Espectro_total_s1[3:])

ruido_C_espectro = np.array(datos_C_espectro[:3])
medidas_C_espectro = np.array(datos_C_espectro[3:])

ruido_A_promed_espectro = np.round(10 * np.log10((1/len(ruido_A_espectro)) * np.sum(10**(0.1*ruido_A_espectro), axis = 0)), 1)
ruido_B_promed_espectro = np.round(10 * np.log10((1/len(ruido_B_espectro)) * np.sum(10**(0.1*ruido_B_espectro), axis = 0)), 1)
ruido_C_promed_espectro = np.round(10 * np.log10((1/len(ruido_C_espectro)) * np.sum(10**(0.1*ruido_C_espectro), axis = 0)), 1)

#print(ruido_A_promed,ruido_B_promed, ruido_C_promed) #TUDO VA BEM

#print(len(medidas_A))
#AHORA VOY A SEPARAR LAS FASES

fases_A_espectro = np.split(medidas_A_espectro,7) #En este array están las medidas de cada fase, SIN PROMEDIAR
fases_B_espectro = np.split(medidas_B_espectro,7) #En este array están las medidas de cada fase, SIN PROMEDIAR
fases_C_espectro = np.split(medidas_C_espectro,7) #En este array están las medidas de cada fase, SIN PROMEDIAR


#print(fases_A[0]) Te devuelve los tres valores de la FASE 1 en la posición A

fases_A_promed_espectro = np.array([
    np.round(10 * np.log10((1 / len(fase)) * np.sum(10**(0.1 * fase), axis = 0)), 1)
    for fase in fases_A_espectro
])

fases_B_promed_espectro = np.array([
    np.round(10 * np.log10((1 / len(fase)) * np.sum(10**(0.1 * fase), axis = 0)), 1)
    for fase in fases_B_espectro
])

fases_C_promed_espectro = np.array([
    np.round(10 * np.log10((1 / len(fase)) * np.sum(10**(0.1 * fase), axis = 0)), 1)
    for fase in fases_C_espectro
])

#TODO: COMPROBAR EL CRITERIO EN CADA FASE DE QUE EL RUIDO DE FONDO SEA 3dB INFERIOR A LA MEDIDA DE LA FASE. SE REALIZA SOBRE EL PROMEDIADO DE LAS TRES MEDIDAS EN CADA PUNTO
#EJEMPLO : FASES_A_PROMED[0]-RUIDO_A_PROMED PARA COMPROBAR EN LA FASE 1
#A INVESTIGAR ANTES: ES POSIBLE QUE HAYA QUE HACER ANTES EL PROMEDIADO DE LOS PUNTOS ABC PARA COMPROBAR ESTO CONTRA EL RUIDO ABC? LA INTUICION ME DICE QUE NO, QUE ES PUNTO A PUNTO

#Comprobación de cada punto, cada fase si cumple el criterio de los 3dB

ruido_promed_espectro = np.round(10 * np.log10((1/3) * ((10**(0.1*ruido_A_promed_espectro)) + (10**(0.1*ruido_B_promed_espectro)) + (10**(0.1*ruido_C_promed_espectro)))), 1)
fases_promed_espectro = np.round(10 * np.log10((1/3) * ((10**(0.1*fases_A_promed_espectro)) + (10**(0.1*fases_B_promed_espectro)) + (10**(0.1*fases_C_promed_espectro)))), 1)

diferencias_espectro = fases_promed_espectro - ruido_promed_espectro
print("La matriz de fases en espectro tiene la forma",{diferencias_espectro.shape})

num_filas, num_cols = fases_promed_espectro.shape
fases_corregidas_espectro = np.zeros((num_filas, num_cols))

num_filas, num_cols = fases_promed_espectro.shape
fases_corregidas_espectro = np.zeros((num_filas, num_cols))

for i in range(num_filas):
    # Fila actual de la fase (30 valores)
    fase_i = fases_promed_espectro[i]
    
    # CORRECCIÓN CLAVE AQUÍ: 
    # El ruido es el mismo vector para todas las fases. No usamos [i].
    ruido_i = ruido_promed_espectro 
    
    # Diferencia precalculada para esta fila
    diff_i = diferencias_espectro[i]
    
    # Copia base
    fila_corregida = fase_i.copy()
    
    # Índices a corregir en esta fila
    indices_a_corregir = np.where(diff_i > 3)[0]
    
    if len(indices_a_corregir) > 0:
        S = fase_i[indices_a_corregir]
        N = ruido_i[indices_a_corregir] # Ahora ruido_i es un vector, así que esto funciona
        
        # Fórmula
        potencia_lineal = (10**(0.1 * S)) - (10**(0.1 * N))
        potencia_lineal = np.maximum(potencia_lineal, 1e-10)
        resultado = np.round(10 * np.log10(potencia_lineal), 1)
        
        fila_corregida[indices_a_corregir] = resultado
    
    fases_corregidas_espectro[i, :] = fila_corregida

print("\nForma final corregida:", fases_corregidas_espectro.shape)

#  REPORTE DE CUMPLIMIENTO (Bandas > 3dB)
print("\n--- Bandas VÁLIDAS por Fase ---")
for i in range(7):
    # Usamos máscara booleana para filtrar las etiquetas directamente
    validas = etiquetas_freq[diferencias_espectro[i] >= 3] 
    print(f"Fase {i+1} CUMPLE en: {list(validas)}")




Intentando leer el archivo: P3_G7_S1_Datos.xlsx...
Éxito: Se leyó un array de 24 filas x 30 columnas.
Intentando leer el archivo: P3_G7_S2_Datos.xlsx...


/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Éxito: Se leyó un array de 48 filas x 30 columnas.
La matriz de fases en espectro tiene la forma {(7, 30)}

Forma final corregida: (7, 30)

--- Bandas VÁLIDAS por Fase ---
Fase 1 CUMPLE en: ['400', '500', '630', '800', '1k', '1.25k', '1.6k', '2k', '2.5k', '3.15k', '4k', '5k', '6.3k', '8k', '10k', '12.5k']
Fase 2 CUMPLE en: ['100', '250', '315', '400', '500', '630', '800', '1k', '1.25k', '1.6k', '2k', '2.5k', '3.15k', '4k', '5k', '6.3k', '8k', '10k', '12.5k']
Fase 3 CUMPLE en: ['200', '250', '315', '400', '500', '630', '800', '1k', '1.25k', '1.6k', '2k', '2.5k', '3.15k', '4k', '5k', '6.3k', '8k', '10k', '12.5k']
Fase 4 CUMPLE en: ['250', '315', '400', '500', '630', '800', '1k', '1.25k', '1.6k', '2k', '2.5k', '3.15k', '4k', '5k', '6.3k', '8k']
Fase 5 CUMPLE en: ['500', '800', '1k', '1.25k', '1.6k', '2k', '2.5k', '3.15k', '4k', '5k', '6.3k', '8k', '10k', '12.5k']
Fase 6 CUMPLE en: ['1.6k']
Fase 7 CUMPLE en: ['500']


Cálculo de Incertidumbre

In [7]:
### CÁLCULO COMPONENTE TIPO A INCERTIDUMBRE DE La

n = 3 # Veces que se repite la medida
f = 7 # Fases
media_arit_A = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0])
media_arit_B = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0])
media_arit_C = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0])
i = 0
j = 0

# Cálculo de la media aritmética en cada punto de cada fase
while j != f:
  while i != n:
    media_arit_A[j] = media_arit_A[j] + (fases_A[j][i])/3
    media_arit_B[j] = media_arit_B[j] + (fases_B[j][i])/3
    media_arit_C[j] = media_arit_C[j] + (fases_C[j][i])/3
    i = i + 1
  i = 0
  j = j + 1
j = 0

print("La media aritmética de cada fase en el punto A es: ", media_arit_A)
print("La media aritmética de cada fase en el punto B es: ", media_arit_B)
print("La media aritmética de cada fase en el punto C es: ", media_arit_C)

# Varianza experimental de las observaciones
var_A = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0])
var_B = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0])
var_C = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0])

while j != f:
  while i != n:
    var_A[j] = var_A[j] + (1/(n-1))*(fases_A[j][i]-media_arit_A[j])*2
    var_B[j] = var_B[j] + (1/(n-1))*(fases_B[j][i]-media_arit_B[j])*2
    var_C[j] = var_C[j] + (1/(n-1))*(fases_C[j][i]-media_arit_C[j])*2
    i = i + 1
  i = 0
  j = j + 1
j = 0

print("La varianza experimental de cada fase en el punto A es: ", var_A)
print("La varianza experimental de cada fase en el punto B es: ", var_B)
print("La varianza experimental de cada fase en el punto C es: ", var_C)

# Desviación típica experimetal (incertidumbre típica de medida ua) de la medida

s_A = np.sqrt(var_A/n)
s_B = np.sqrt(var_B/n)
s_C = np.sqrt(var_C/n)

print("La incertidumbre típica de medida de cada fase en el punto A es: ", s_A)
print("La incertidumbre típica de medida de cada fase en el punto B es: ", s_B)
print("La incertidumbre típica de medida de cada fase en el punto C es: ", s_C)


### CÁLCULO COMPONENTES TIPO B

## INCERTIDUMBRE DE La

# Selección perceptual de los 3 posiciones de medida de “peor caso”. --> Variación de 1.5dB como mucho
u_sel = 1.5/np.sqrt(3)
# De la practica, se asume: 94 +- 0,3 dB -> +- 0,3 dB con k=2
u_slm = 0.3 / 2.0
# Correcciones ambientales: Incertidumbre tipica de temperatura y presion
T_nom = 20.0
P_nom = 98
u_T = 0.1 / np.sqrt(12) #Incertidumbre por la resolucion de la temperatura (0,1 C)
u_P = 2 / np.sqrt(12) #Incertidumbre por la resolucion de la presion (2 kPa)

def C1_of(T, P):
    return -10*np.log10(P / 101.325) + 5*np.log10((273 + T) / 314)

def C2_of(T, P):
    return -10*np.log10(P / 101.325) + 15*np.log10((273 + T) / 296)

# Aproximacion de derivadas parciales de C1 y C2 en funcion de temperatura y presion
dC1_dT = (C1_of(T_nom + u_T, P_nom) - C1_of(T_nom - u_T, P_nom)) / (2 * u_T)
dC1_dP = (C1_of(T_nom, P_nom + u_P) - C1_of(T_nom, P_nom - u_P)) / (2 * u_P)

dC2_dT = (C2_of(T_nom + u_T, P_nom) - C2_of(T_nom - u_T, P_nom)) / (2 * u_T)
dC2_dP = (C2_of(T_nom, P_nom + u_P) - C2_of(T_nom, P_nom - u_P)) / (2 * u_P)

# Incertidumbres tipica u_C1 y u_C2
u_C1 = np.sqrt((dC1_dT * u_T)**2 + (dC1_dP * u_P)**2)
u_C2 = np.sqrt((dC2_dT * u_T)**2 + (dC2_dP * u_P)**2)
# Consideramos u_C3 despreciable

# Corrección por ruido de fondo resultante en cota
u_cota = ...

## Incertidumbre de Lkx
# Corrección componentes Kt Ki Kf
u_corcomp = ...
# Promediado del nivel de la fase de ruido con 3 puntos
u_prom = 1.0/np.sqrt(3)
# Generalización de los niveles de la actividad (periodo anual)
u_gen  = 1.5/np.sqrt(3)

# Incertidumbre total
u_total = np.sqrt(u_sel**2 + u_slm**2 + u_C1**2 + u_C2**2 + u_cota**2 + u_corcomp**2 + u_prom**2 + u_gen**2)

# # Cálculo de coeficientes para la propagación a L_K
denominador=sum(La_fases*10**(0.1*LK_fases))
coeficientes=np.zeros(10)
for i in range(10):
  coeficientes[i]=(La_fases[i]*(10**(0.1*LK_fases[i]))*(np.log(10))*0.1)/denominador

# Incertidumbre estandar del nivel global
k = 2.0
u_LK_peor_dia = np.sqrt(np.sum((coeficientes*u_total)**2))
U_LK_peor_dia = k * u_LK_peor_dia


La media aritmética de cada fase en el punto A es:  [41.95333333 46.27666667 45.25       43.49666667 41.59333333 39.87
 40.26666667]
La media aritmética de cada fase en el punto B es:  [41.06666667 43.13666667 43.80333333 43.20666667 41.21       40.58
 40.88333333]
La media aritmética de cada fase en el punto C es:  [44.75666667 46.21       47.40666667 42.47       41.20666667 39.19333333
 39.59333333]
La varianza experimental de cada fase en el punto A es:  [ 0.00000000e+00 -1.42108547e-14 -1.42108547e-14  1.42108547e-14
  0.00000000e+00  7.10542736e-15  0.00000000e+00]
La varianza experimental de cada fase en el punto B es:  [1.42108547e-14 7.10542736e-15 0.00000000e+00 7.10542736e-15
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
La varianza experimental de cada fase en el punto C es:  [-7.10542736e-15 -7.10542736e-15 -7.10542736e-15  7.10542736e-15
  7.10542736e-15  1.42108547e-14  0.00000000e+00]
La incertidumbre típica de medida de cada fase en el punto A es:  [0.00000000e+00     

/tmp/ipykernel_30286/64423911.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  media_arit_A[j] = media_arit_A[j] + (fases_A[j][i])/3
/tmp/ipykernel_30286/64423911.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  media_arit_B[j] = media_arit_B[j] + (fases_B[j][i])/3
/tmp/ipykernel_30286/64423911.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  media_arit_C[j] = media_arit_C[j] + (fases_C[j][i])/3
/tmp/ipykernel_30286/64423911.py:33: DeprecationWarning: C

TypeError: unsupported operand type(s) for ** or pow(): 'ellipsis' and 'int'

Declaración de conformidad

In [ ]:
# print(f"L_K,d = {LK_d:.2f} ± {U_LK_d:.2f} dB -> {}")
# print(f"L_K,e = {LK_e:.2f} ± {U_LK_e:.2f} dB -> {}")
# print(f"L_K,n = {LK_n:.2f} ± {U_LK_n:.2f} dB -> {}")